In [19]:
import gradio as gr
import pandas as pd
import joblib
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from utils import set_label_encoding
from datetime import datetime
from tabulate import tabulate

In [20]:
# labels - column to use in resulting from model and used in gradio
target_labels = ['Amoxicilline', 'Augmentin', 'Oxacilline / cefazoline', 'Tazocilline', 'Cefotaxime / ceftriaxone', 'Ceftazidime', 'Cefepime', 'Aztreonam', 'Imipenem', 'Meropenem', 'Ertapenem', 'Amikacine', 'Gentamicine', 'Ciprofloxacine', 'Levofloxacine', 'Bactrim', 'Vancomycine','Rifampicine','Clindamycine''Macrolides']

In [21]:
# pip install tabulate

In [22]:
df = pd.read_excel('dataset/Calmette_data.xlsx')
# df = df.applymap(lambda x: x.replace('\u200b', '') if isinstance(x, str) else x)
df = df.replace(to_replace='\u200b', value='', regex=True)
# drop columns
df = df.drop(columns=["Unnamed: 0", "Hopital", "IDLabo", "IDPatient", "visittype"])
df = df.rename(columns={"new_age": "age"})
# Convert column names to lowercase
df.columns = df.columns.str.lower()
# Mapping Khmer to English using map()
df['sex'] = df['sex'].map({'ប្រុស': 'Male', 'ស្រី': 'Female'})

In [23]:
# Set Label Encoding to convert features to number
age_dict = set_label_encoding(df, 'age')
sex_dict = set_label_encoding(df, 'sex')
service_code_dict = set_label_encoding(df, 'service_code')
service_type_dict = set_label_encoding(df, 'service_type')
prelevement_type_dict = set_label_encoding(df, 'prelevement_type')
germe_dict = set_label_encoding(df, 'germe')
espece_requete_dict = set_label_encoding(df, 'espece_requete')
direct_2_dict = set_label_encoding(df, '2_direct')
culture_3_dict = set_label_encoding(df, '3_culture')
genre_4_dict = set_label_encoding(df, '4_genre')
espece_5_training_dict = set_label_encoding(df, '5_espece_training')
contamination_dict = set_label_encoding(df, 'contamination')

In [24]:
def amr_project(age, sex, service_code, service_type, date, prelevement_type, germe, contamination, direct_2, culture_3, genre_4, species_training_5):
    print(sex, sex_dict[sex])
    print(service_code, service_code_dict[service_code])
    print(service_type, service_type_dict[service_type])

    # Convert timestamp to a datetime object
    date_time = datetime.fromtimestamp(date)
    # Extract the month
    month = date_time.month
    print(date, month)

    print(prelevement_type, prelevement_type_dict[prelevement_type])
    print(germe, germe_dict[germe])

    contamination = 0 if contamination == 'No' else 1 # Convert contamination to 0 and 1
    print(contamination)

    print(direct_2, direct_2_dict[direct_2])
    print(culture_3, culture_3_dict[culture_3])
    print(genre_4, genre_4_dict[genre_4])
    print(species_training_5, espece_5_training_dict[species_training_5])
    print()
    print(age, sex, service_code, service_type, date, prelevement_type, germe, contamination, direct_2, culture_3, genre_4, species_training_5)

    # Sample data
    input_data = {
        "month": [month],
        "age": [age],
        'sex': [sex_dict[sex]],
        'service_code': [service_code_dict[service_code]],
        'service_type': [service_type_dict[service_type]],
        'prelevement_type': [prelevement_type_dict[prelevement_type]],
        'espece_requete': [espece_requete_dict[germe]],
        '2_direct': [direct_2_dict[direct_2]],
        '3_culture': [culture_3_dict[culture_3]],
        '4_genre': [genre_4_dict[genre_4]],
        '5_espece_training': [espece_5_training_dict[species_training_5]],
        'contamination': [contamination]
    }
    
    # Create the DataFrame
    df = pd.DataFrame(input_data)
    print(tabulate(df, headers="keys", tablefmt="fancy_grid"))
    # print(df)

    # Load the model back from the file
    rf_model_loaded = joblib.load('testing_random_forest_model.joblib')
    one_row_test = df

    # Now you can use the loaded model to make predictions
    y_pred_loaded = rf_model_loaded.predict(one_row_test)
    # print(len(y_pred_loaded[0]), np.array(y_pred_loaded))

    y_pred_prob = rf_model_loaded.predict_proba(one_row_test)
    # print(len(y_pred_prob), np.array(y_pred_prob))

    result_class_dict = dict()
    result_probab_dict = dict()

    for i in range(len(target_labels)):
        message = "Sensible" if y_pred_loaded[0][i] == 1 else "Resistance"
        print(target_labels[i], ": class - ", y_pred_loaded[0][i], message,' - Probab:', y_pred_prob[i][0][1])
        result_class_dict[target_labels[i]] = message
        result_probab_dict[target_labels[i]] = y_pred_prob[i][0][1]

    print()
    print(result_class_dict)
    print(result_probab_dict)

    # Sample data
    data = {
        "Model": ["Randome Forest"]
    }
    # Create the DataFrame
    df_info = pd.DataFrame(data)
    print(tabulate(df_info, headers="keys", tablefmt="fancy_grid"))

    return df_info, result_probab_dict
    # return df_info, {'Amoxicilline': 0.74, 'Augmentin': 0.22, 'Oxacilline / cefazoline': 0.62, 'Tazocilline': 0.11, 'Cefotaxime / ceftriaxone': 0.94, 'Ceftazidime': 0.38, 'Cefepime': 0.6, 'Aztreonam': 0.5, 'Imipenem': 0.71, 'Meropenem': 0.12, 'Ertapenem': 0.58, 'Amikacine': 0.22, 'Gentamicine': 0.87, 'Ciprofloxacine': 0.18, 'Levofloxacine': 0.85, 'Bactrim': 0.96, 'Vancomycine': 0.7, 'Rifampicine': 0.16, 'Clindamycine': 0.85, 'Macrolides': 0.89}



app = gr.Interface(fn=amr_project, inputs=[
    gr.Textbox(label="Age"),
    gr.Radio(["Male", "Female"], label="Sex"),
    gr.Dropdown(['ms e1', 'mat 1', 'mat 3', 'ms', 'ms e6', 'chirurgie b', 'med b', 'med a4', 'ncc e5', 'urgence central', 'chirurgie a', 'med a2', 'med a5', 'gynecologie', 'neuro-chirurgie', 'med a1', 'neuro-medicine', 'ncc e1', 'nch3', 'chirurgie a2 e2', 'ns4', 'chirugie a e1', 'chirurgie a2 e1', 'ncc e2', 'ncc e4', 'pe', 'rmc', 'med a6', 'med a3', 'pve', 'cardio b', 'rns', 'ncc e3', 'hepato-gastro enterologie', 'réa b', 'med a7', 'c.gy', 'c. gy bs2', 'rea-neurologie', 'ma-ta', 'cardio c', 'cardio a', 'urgence porte', 'ma-ta s.n', 'ma-ta s.i', 'ma-ta s.p', 'chir. ta1', 'réa a', 'ma-ta s.e', 'réa c', 'chir. ta2', 'med a1e0', 'gy-ta8', 'med a1e1', 'gy-pma', 'chirurgie a e0'], label="Service Code"),
    gr.Dropdown(['Pediatrie', 'Chirurgie', 'Medecine', 'SAU', 'Gyneco/Obs', 'Rea'], label="Service Type"),
    gr.DateTime(label='Date'),
    gr.Dropdown(['Collection/Abces', 'Hemoc/KT', 'Urine', 'Genital', 'Respiratoire', 'LCR', 'Sereuse', 'Osteoarticulaire', 'Biopsie', 'Digestif', 'Peau', 'Materiel'], label='Prelevement Type'),
    gr.Dropdown(['Streptococcus anginosus', 'Staphylococcus aureus', 'Enterobacter cloacae', 'Pantoea agglomerans', 'Burkholderia cepacia', 'Acinetobacter baumannii', 'Acinetobacter sp.', 'Stenotrophomonas maltophilia', 'Pseudomonas aeruginosa', 'Streptococcus uberis', 'Staphylococcus hominis', 'Staphylococcus epidermidis', 'Enterococcus faecalis', 'Enterobacter sakazakii', 'Streptococcus constellatus', 'Streptococcus intermedius', 'Staphylococcus saprophyticus', 'Enterococcus faecium', 'Klebsiella pneumoniae', 'Proteus mirabilis', 'Raoultella ornithinolytica', 'Candida albicans', 'Streptococcus agalactiae/B', 'Streptococcus mitis', 'Streptococcus acidominimus', 'Streptococcus gordonii', 'Streptococcus gallolyticus', 'Lactococcus garvieae', 'Streptococcus oralis', 'Streptococcus group C/G', 'Streptococcus sp.', 'Streptococcus sanguinis', 'Staphylococcus haemolyticus', 'Salmonella Typhi', 'Salmonella Paratyphi A', 'Salmonella sp.', 'Salmonella choleraesuis', 'Salmonella enterica', 'Enterococcus casseliflavus', 'Enterococcus casseliflavus/gallinarum', 'Enterococcus avium', 'Citrobacter koseri', 'Klebsiella aerogenes', 'Citrobacter freundii', 'Klebsiella ozaenae', 'Proteus vulgaris', 'Klebsiella oxytoca', 'Providencia stuartii', 'Morganella morganii', 'Citrobacter youngae', 'Enterococcus durans', 'Burkholderia pseudomallei', 'Streptococcus suis', 'Streptococcus pyogenes/A', 'Aeromonas hydrophila', 'Aeromonas caviae', 'Aeromonas veronii', 'Acinetobacter lwoffii', 'Acinetobacter haemolyticus', 'Streptococcus dysgalactiae', 'Streptococcus parasanguinis', 'Staphylococcus lugdunensis', 'Staphylococcus caprae', 'Streptococcus pneumoniae', 'Enterococcus gallinarum', 'Enterococcus hirae', 'Enterococcus sp.', 'Candida glabrata', 'Candida tropicalis', 'Candida melibiosica', 'Aeromonas sobria', 'Aerococcus viridans', 'Pseudomonas putida', 'Enterococcus raffinosus', 'Serratia liquefaciens', 'Edwardsiella tarda', 'Plesiomonas shigelloides', 'Listeria monocytogenes', 'Streptococcus C', 'Candida sp.', 'Candida parapsilosis', 'Gemella morbillorum', 'Streptococcus vestibularis', 'Streptococcus porcinus', 'Staphylocoque a coagulase negative', 'Enterobacter aerogenes', 'Citrobacter farmeri', 'Raoultella terrigena', 'Serratia marcescens', 'Acinetobacter lwoffii/haemolyticus', 'Providencia rettgeri', 'Shigella sonnei', 'Cronobacter sakazakii'], label='Germe'),
    gr.Radio(['Yes', 'No'], label='Contamination'),
    gr.Dropdown(['Cocci Gram Pos', 'Bacille Gram Neg', 'Levure', 'Bacille Gram Pos'], label='Stage 2 - Direct'),
    gr.Dropdown(['Cocci Gram Pos type streptocoque', 'Cocci Gram Pos type staphylocoque', 'Bacille Gram Neg type enterobacterie', 'Bacille Gram Neg non fermentant', 'Cocci Gram Pos type enterocoque', 'Levure', 'Cocci Gram Pos non specifie', 'Bacille Gram Neg non specifie', 'Bacille Gram Pos'], label='Stage 3 - Culture'),
    gr.Dropdown(['Streptococcus', 'Staphylococcus', 'Enterobacter', 'Pantoea', 'Burkholderia', 'Acinetobacter', 'Stenotrophomonas', 'Pseudomonas', 'Enterococcus', 'Klebsiella', 'Proteus', 'Candida', 'Lactococcus', 'Salmonella', 'Citrobacter', 'Providencia', 'Morganella', 'Aeromonas', 'Aerococcus', 'Serratia', 'Edwardsiella', 'Plesiomonas', 'Listeria', 'Gemella', 'Shigella', 'Cronobacter'], label='Stage 4 - Genre'),
    gr.Dropdown(['Streptococcus anginosus/constellatus/intermedius/milleri', 'Staphylococcus aureus', 'Enterobacter cloacae complex', 'Pantoea agglomerans', 'Burkholderia cepacia complex', 'Acinetobacter baumannii-calcoaceticus complex', 'Acinetobacter sp.', 'Stenotrophomonas maltophilia', 'Pseudomonas aeruginosa', 'Streptococcus uberis', 'Staphylococcus hominis', 'Staphylococcus epidermidis', 'Enterococcus faecalis', 'Cronobacter sakazakii', 'Staphylococcus saprophyticus', 'Enterococcus faecium', 'Klebsiella pneumoniae', 'Proteus mirabilis', 'Klebsiella ornithinolytica', 'Candida albicans', 'Streptococcus agalactiae/B', 'Streptococcus mitis/australis/cristatus/infantis/massiliensis/oligofermentans/oralis/peroris/pseudopneumoniae/sinensis', 'Streptococcus acidominimus', 'Streptococcus sanguinis/parasanguinis/gordonii', 'Streptococcus equinus(bovis)/gallolyticus(caprinus)/infantarius/lutetiensis/alactolyticus/pasteurianus/D(hors enterococoques)', 'Lactococcus garvieae', 'Streptococcus dysgalactiae/equi/equisimilis/C/canis/G', 'Streptococcus sp.', 'Staphylococcus haemolyticus', 'Salmonella Typhi', 'Salmonella Paratyphi', 'Salmonella sp.', 'Salmonella enterica', 'Enterococcus casseliflavus/gallinarum', 'Enterococcus avium', 'Citrobacter koseri', 'Klebsiella aerogenes', 'Citrobacter freundii complex', 'Klebsiella ozaenae', 'Proteus vulgaris', 'Klebsiella oxytoca', 'Providencia stuartii', 'Morganella morganii', 'Enterococcus durans', 'Burkholderia pseudomallei', 'Streptococcus suis', 'Streptococcus pyogenes/A', 'Aeromonas caviae/dhakensis/eucrenophila/hydrophila/punctata/sobria/veronii', 'Acinetobacter lwoffii/haemolyticus', 'Staphylococcus lugdunensis', 'Staphylococcus caprae', 'Streptococcus pneumoniae', 'Enterococcus gallinarum', 'Enterococcus hirae', 'Enterococcus sp.', 'Candida glabrata', 'Candida tropicalis', 'Candida melibiosica', 'Aerococcus viridans', 'Pseudomonas putida', 'Enterococcus raffinosus', 'Serratia liquefaciens', 'Edwardsiella tarda', 'Plesiomonas shigelloides', 'Listeria monocytogenes', 'Candida sp.', 'Candida parapsilosis', 'Gemella morbillorum', 'Streptococcus salivarius/thermophilus/vestibularis', 'Streptococcus porcinus', 'Staphylocoque a coagulase negative', 'Citrobacter amalonaticus group', 'Klebsiella terrigena', 'Serratia marcescens', 'Providencia rettgeri', 'Shigella sonnei', 'Cronobacter malonaticus'], label='Stage 5 - Species Training')
    # gr.Dropdown(['Amoxicilline', 'Augmentin', 'Oxacilline / cefazoline', 'Tazocilline', 'Cefotaxime / ceftriaxone', 'Ceftazidime', 'Cefepime', 'Aztreonam', 'Imipenem', 'Meropenem', 'Ertapenem', 'Amikacine', 'Gentamicine', 'Ciprofloxacine', 'Levofloxacine', 'Bactrim', 'Vancomycine', 'Rifampicine', 'Clindamycine', 'Macrolides'], label='Antibiotics')
    ], 
    outputs=[
        gr.Dataframe(label='Information'),
        gr.Label(label='Antibiotics - Sensibility Prediction')
        ]
    )

app.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


/Users/thearsophal/opt/miniconda3/lib/python3.9/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Male 1
réa a 51
Medecine 2
1733814837.0 12
Osteoarticulaire 7
Stenotrophomonas maltophilia 99
1
Bacille Gram Neg 0
Bacille Gram Neg non fermentant 0
Providencia 18
Citrobacter freundii complex 14

30 Male réa a Medecine 1733814837.0 Osteoarticulaire Stenotrophomonas maltophilia 1 Bacille Gram Neg Bacille Gram Neg non fermentant Providencia Citrobacter freundii complex
╒════╤═════════╤═══════╤═══════╤════════════════╤════════════════╤════════════════════╤══════════════════╤════════════╤═════════════╤═══════════╤═════════════════════╤═════════════════╕
│    │   month │   age │   sex │   service_code │   service_type │   prelevement_type │   espece_requete │   2_direct │   3_culture │   4_genre │   5_espece_training │   contamination │
╞════╪═════════╪═══════╪═══════╪════════════════╪════════════════╪════════════════════╪══════════════════╪════════════╪═════════════╪═══════════╪═════════════════════╪═════════════════╡
│  0 │      12 │    30 │     1 │             51 │              2 │     

/Users/thearsophal/opt/miniconda3/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/thearsophal/opt/miniconda3/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Amoxicilline : class -  0 Resistance  - Probab: 0.06
Augmentin : class -  0 Resistance  - Probab: 0.13
Oxacilline / cefazoline : class -  0 Resistance  - Probab: 0.025
Tazocilline : class -  0 Resistance  - Probab: 0.387
Cefotaxime / ceftriaxone : class -  0 Resistance  - Probab: 0.205
Ceftazidime : class -  0 Resistance  - Probab: 0.475
Cefepime : class -  0 Resistance  - Probab: 0.40700000000000003
Aztreonam : class -  0 Resistance  - Probab: 0.165
Imipenem : class -  0 Resistance  - Probab: 0.405
Meropenem : class -  1 Sensible  - Probab: 0.56
Ertapenem : class -  0 Resistance  - Probab: 0.24
Amikacine : class -  0 Resistance  - Probab: 0.37200000000000005
Gentamicine : class -  0 Resistance  - Probab: 0.37
Ciprofloxacine : class -  0 Resistance  - Probab: 0.287
Levofloxacine : class -  0 Resistance  - Probab: 0.152
Bactrim : class -  0 Resistance  - Probab: 0.36
Vancomycine : class -  0 Resistance  - Probab: 0.01
Rifampicine : class -  0 Resistance  - Probab: 0.085
ClindamycineMacr